In [5]:
options(java.parameters = "-Xmx50g") 
library(loadeR)
library(transformeR)
library(visualizeR)
library(climate4R.UDG)
library(dplyr)
library(climate4R.value)
library(VALUE)
library(downscaleR) # para bias correction (función biasCorrection)
library(climate4R.indices) # para calcular índices

# Inventario con lista de datasets disponibles
df <- read.csv("https://data.meteo.unican.es/inventory.csv")
# df <- read.csv("../Data_AEMET/inventory.csv")

Warning message:
"package 'loadeR' was built under R version 3.6.3"Loading required package: rJava
Warning message:
"package 'rJava' was built under R version 3.6.3"Loading required package: loadeR.java
Warning message:
"package 'loadeR.java' was built under R version 3.6.3"Java version 21x amd64 by Azul Systems, Inc. detected
NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready
Loading required package: climate4R.UDG
Warning message:
"package 'climate4R.UDG' was built under R version 3.6.3"climate4R.UDG version 0.2.6 (2023-06-26) is loaded
Please use 'citation("climate4R.UDG")' to cite this package.
loadeR version 1.8.1 (2023-06-22) is loaded
Please use 'citation("loadeR")' to cite this package.
Warning message:
"package 'transformeR' was built under R version 3.6.3"


    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded
Please see 'citation("transformeR")' to cite this package.
Warning message:
"package 'visualizeR' was built under R version 3.6.3"visualizeR version 1.6.4 (2023-10-26) is loaded
Please see 'citation("visualizeR")' to cite this package.
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'climate4R.value' was built under R version 3.6.3"Loading required package: VALUE
Warning message:
"package 'VALUE' was built under R version 3.6.3"---------------------------------------------- 
| VALUE version 2.2.4 (2023-06-22) is loaded |
|         http://www.value-cost.eu           |
----------------------------------------------
Warning message:
"package 'downscaleR' was built under R version 3.6.3"downscaleR versio

In [27]:
#############################
# *** Leer observaciones ****
#############################
obs.data <- readRDS("obs.pr.rds")


In [28]:
#########################
# *** Leer un modelo ****
#########################
rcm.hist.subset <- subset(df, activity == 'CORDEX' & domain=='EUR-11' & experiment == 'historical' & variable=='pr' & rcm =='RACMO22E' & model == 'ICHEC-EC-EARTH' & ensemble == 'r1i1p1') 
rcm.hist.loc <- as.character(rcm.hist.subset $location)
hist.data <- loadGridData(rcm.hist.loc, var="pr", years= 1986:2005,lonLim =c(-10, 5), latLim = c(35,44))
# Comprobar las unidades de los datos
message("model units ",getGridUnits(hist.data)) # degC para temperaturas, mm ó kg*m-2 para precipitación
# Pasar a grados Celsius
hist.data <- gridArithmetics(hist.data, 273.15, operator="-")
hist.data <- gridArithmetics(hist.data, 86400, operator="*")

[2024-04-26 11:39:34] Opening dataset...
[2024-04-26 11:39:37] The dataset was successfuly opened
[2024-04-26 11:39:37] Defining geo-location parameters
[2024-04-26 11:39:39] Defining time selection parameters
[2024-04-26 11:39:39] Retrieving data subset ...
[2024-04-26 11:43:37] Done
model units K


ERROR: Error: no se puede ubicar un vector de tamaño  758.4 Mb


In [ ]:
##############################################
# *** Aplicar máscara de tierra al modelo ****
##############################################
rcm.mask <- subset(df, activity == 'CORDEX' & domain=='EUR-11' & experiment == 'historical' & variable=='sftlf' & rcm =='RACMO22E' & model == 'ICHEC-EC-EARTH' & ensemble == 'r1i1p1') 
rcm.mask.loc <- as.character(rcm.mask$location)
rcm.mask <- loadGridData(rcm.mask.loc, var="sftlf", lonLim =c(-10, 5), latLim = c(35,44))
# En este modelo, valores entre 0 y 100 de land area fraction, establecemos un umbral para decir qué es tierra y qué es mar

rcm.mask$Data[rcm.mask$Data < 40] <- NA
rcm.mask$Data[rcm.mask$Data >= 40] <- 1 # OJO! puede que en el algún modelo venga en tanto por 1 en lugar de %. Adaptar.

# Multiplicamos los datos del modelo por su máscara tierra-mar. Pondrá NA en los puntos de mar.
# Se hace una multiplicación día a día de los datos del modelo por la máscara.
time <- getRefDates(hist.data)
nt <- length(time)
ls <-lapply(1:nt, function(i){
  timei <-subsetDimension(hist.data, dimension = "time", indices=i) # separa día a día 
  gridArithmetics(timei, rcm.mask, operator = "*") # multiplica los dos grids
}
)
rcm.hist.masked <- bindGrid(ls, dimension = "time") # bindGrid es una función de transforeR que vuelve a unir todos los días en un grid de climate4R
rm(hist.data, ls, rcm.mask)

In [ ]:
############################################################
# *** Interpolar el modelo a la malla de la observación ****
############################################################
# Este paso lo hace automáticamente biasCorrection, pero lo hago aquí para representar el modelo en la misma malla que la observación y calcular el sesgo (bias)
rcm.hist.interp <- interpGrid(rcm.hist.masked, new.coordinates = getGrid(obs.data))

In [ ]:
png("interpolacion_pr_EUR-11.png")
spatialPlot(makeMultiGrid(climatology(obs.data),climatology(rcm.hist.interp)) , backdrop.theme = "coastline", 
            rev.colors = TRUE,  at= seq(0,25), main= "Precipitación en Iberia", as.table=TRUE, 
            names.attr=c("OBS", "RCM interpolado"), layout=c(2,1)) # pinta ambos en las latitudes que tocan
dev.off()

In [ ]:
##########################
# *** Bias correction ****
##########################
bc.scaling <- biasCorrection(y=obs.data, x=rcm.hist.interp, newdata= rcm.hist.interp, precipitation = FALSE, 
                             method="scaling", scaling.type="additive" ) # correción solo de la media (aditiva para temperatura)
bc.eqm <- biasCorrection(y=obs.data, x=rcm.hist.interp, newdata= rcm.hist.interp, precipitation = FALSE, 
                         method="eqm", extrapolation = "constant", n.quantiles=99 )  # corrección de 99 percentiles

In [ ]:
# Calcular el sesgo de los datos sin corregir (raw) y de las dos correcciones en la media
bias.raw.mean <- gridArithmetics(climatology(rcm.hist.interp), climatology(obs.data), operator="-")
bias.scaling.mean <- gridArithmetics(climatology(bc.scaling), climatology(obs.data), operator="-")
bias.eqm.mean <- gridArithmetics(climatology(bc.eqm), climatology(obs.data), operator="-")

In [ ]:
###########################
### Sesgo RCM Historico ###
###########################
png("sesgo_pr_EUR-11.png", width = 1300, height = 800)
spatialPlot(makeMultiGrid(bias.raw.mean, bias.scaling.mean, bias.eqm.mean), backdrop.theme = "coastline", 
            rev.colors = TRUE,  at= seq(-6,6), main= "Sesgo en la Precipitación (promedio)", layout=c(3,1),
            as.table=TRUE, names.attr=c("RAW", "BC (scaling)", "BC (eqm)"))
dev.off()

In [ ]:
###################
### Save Sesgos ###
###################
saveRDS(bias.raw.mean, "sesgo_pr_EUR-11_raw.rds", compress = "xz")
saveRDS(bias.scaling.mean, "sesgo_pr_EUR-11_scaling.rds", compress = "xz")
saveRDS(bias.eqm.mean, "sesgo_pr_EUR-11_eqm.rds", compress = "xz")

In [ ]:
###############################################
# *** Bias correction del escenario futuro ****
###############################################
# TAREA: Hacer lo mismo, pero esta vez aplicar la correción a las simulaciones de futuro (argumento newdata)
rcm.rcp85.subset <- subset(df, activity == 'CORDEX' & domain=='EUR-11' & experiment == 'rcp85' & variable=='pr' & rcm =='RACMO22E' & model == 'ICHEC-EC-EARTH' & ensemble == 'r1i1p1') 
rcm.rcp85.loc <- as.character(rcm.rcp85.subset $location)
rcp85.data <- loadGridData(rcm.rcp85.loc, var="pr", years= 2041:2070,lonLim =c(-10, 5), latLim = c(35,44))
# Comprobar las unidades de los datos
message("model units ",getGridUnits(rcp85.data)) # degC para temperaturas, mm ó kg*m-2 para precipitación
# Pasar a grados Celsius
rcp85.data <- gridArithmetics(rcp85.data, 86400, operator="*")

In [ ]:
##############################################
# *** Aplicar máscara de tierra al modelo ****
##############################################
rcm.mask <- subset(df, activity == 'CORDEX' & domain=='EUR-11' & experiment == 'historical' & variable=='sftlf' & rcm =='RACMO22E' & model == 'ICHEC-EC-EARTH' & ensemble == 'r1i1p1') 
rcm.mask.loc <- as.character(rcm.mask$location)
rcm.mask <- loadGridData(rcm.mask.loc, var="sftlf", lonLim =c(-10, 5), latLim = c(35,44))
# En este modelo, valores entre 0 y 100 de land area fraction, establecemos un umbral para decir qué es tierra y qué es mar

rcm.mask$Data[rcm.mask$Data < 40] <- NA
rcm.mask$Data[rcm.mask$Data >= 40] <- 1 # OJO! puede que en el algún modelo venga en tanto por 1 en lugar de %. Adaptar.

# Multiplicamos los datos del modelo por su máscara tierra-mar. Pondrá NA en los puntos de mar.
# Se hace una multiplicación día a día de los datos del modelo por la máscara.
time <- getRefDates(rcp85.data)
nt <- length(time)
ls <-lapply(1:nt, function(i){
  timei <-subsetDimension(rcp85.data, dimension = "time", indices=i) # separa día a día 
  gridArithmetics(timei, rcm.mask, operator = "*") # multiplica los dos grids
}
)
rcp85.masked <- bindGrid(ls, dimension = "time") # bindGrid es una función de transforeR que vuelve a unir todos los días en un grid de climate4R
rm(rcp85.data, ls, rcm.mask)

In [ ]:
############################################################
# *** Interpolar el modelo a la malla de la observación ****
############################################################
# Este paso lo hace automáticamente biasCorrection, pero lo hago aquí para representar el modelo en la misma malla que la observación y calcular el sesgo (bias)
rcp85.interp <- interpGrid(rcp85.masked, new.coordinates = getGrid(obs.data))

In [ ]:
png("interpolacion_pr_EUR-11_rcp85_4170.png", width = 1500, height = 700)
spatialPlot(makeMultiGrid(climatology(obs.data),climatology(rcp85.interp), skip.temporal.check = TRUE) , backdrop.theme = "coastline", 
            rev.colors = TRUE,  at= seq(0,25), main= "Precip en Iberia", as.table=TRUE, 
            names.attr=c("OBS", "RCM interpolado"), layout=c(2,1)) # pinta ambos en las latitudes que tocan
dev.off()

In [ ]:
saveRDS(rcp85.interp, "rcp85_interpolado_pr_EUR-11_41_70_bueno.rds", compress = "xz")

In [14]:
sesgo.scaling <- readRDS("EUR-11/sesgo_pr_EUR-11_scaling.rds")
sesgo.eqm <- readRDS("EUR-11/sesgo_pr_EUR-11_eqm.rds")

In [9]:
futuro <- readRDS("EUR-11/rcp85_interpolado_pr_EUR-11_71_100.rds")

In [17]:
futuro_scaling <- futuro
futuro_eqm <- futuro
for (i in dim(futuro)[1]){
    futuro_scaling$Data[i,,] <- gridArithmetics(futuro$Data[i,,], sesgo.scaling$Data, operator = "+")
}

In [19]:
futuro_scaling <- futuro
dimension <- dim(futuro$Data)
for (i in 1:dimension[1]){
    for (j in 1:dimension[2]){
        for (k in 1:dimension[3]){
            futuro_scaling$Data[i,j,k] <- gridArithmetics(futuro_scaling$Data[i,j,k], sesgo.scaling$Data[1,j,k], operator = "+")
        }
    }
}

ERROR: Error: $ operator is invalid for atomic vectors
